In [1]:
from astropy.stats import mad_std

from ffs import FFS

from astropy.io import fits
import os
import numpy as np

import time

path = "/work/vela/oca/fits/zb08/raw"
#path = "/work/vela/oca/fits/zb08/processed-ofp/science"
noce = [d for d in os.listdir(path) if (os.path.isdir(path+"/"+d) and not os.path.islink(path+"/"+d) )]

noce = sorted(noce)

print(noce)

['0088', '0091', '0095', '0096', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0108', '0112', '0113', '0114', '0116', '0117', '0119', '0154', '0156', '0157', '0161', '0163', '0164', '0165', '0166', '0167', '0168', '0169', '0170', '0171', '0172', '0173', '0174', '0175', '0176', '0177', '0178', '0179', '0180', '0181', '0182', '0183', '0184', '0185', '0186', '0187', '0188', '0189', '0190', '0191', '0192', '0193', '0194', '0195', '0196', '0197', '0198', '0199', '0200', '0201', '0202', '0203', '0204', '0205', '0206', '0208', '0209', '0210', '0211', '0214', '0215', '0216', '0217', '0219', '0220', '0221', '0222', '0223', '0224', '0225', '0226', '0227', '0228', '0229', '0231', '0232', '0233', '0234', '0235', '0236', '0237', '0238', '0239', '0240', '0241', '0242', '0243', '0248', '0249', '0250', '0251', '0252', '0253', '0254', '0255', '0256', '0257', '0258', '0260', '0261', '0262', '0263', '0264', '0265', '0266', '0267', '0268', '0269', '0270', '0271', '0272', '0273',

In [2]:
fits_list = []
night = "1030"
for f in os.listdir(path+"/"+night):
    if os.path.isfile(path+"/"+night+"/"+f):
        fits_list.append(f)

fits_list = sorted(fits_list)
print(len(fits_list))

945


In [3]:
for f in fits_list[100:110]:
    t0 = time.time()
    hdul = fits.open(path+"/"+night+"/"+f)
    data = hdul[0].data
    hdr = hdul[0].header
    hdul.close()
    t1 = time.time()

    if hdr["IMAGETYP"] == "science":
        t2 = time.time()
        ffs = FFS(data)
        t3 = time.time()
        ffs.mk_stats()
        t4 = time.time()
        ffs.sky_gradient()
        t5 = time.time()
        ffs.find_stars(threshold=5,kernel_size=10,fwhm=3)
        t6 = time.time()
        #fwhm_x,fwhm_y = ffs.fwhm(radius=15, all_stars=False)
        t7 = time.time()
        maska = data > np.median(data) + 3 * mad_std(data)
        line_val, line_tht, line_r = ffs.hough_transform(maska)
        t8 = time.time()
        fwx,fwy,ell,th,cpe_t = ffs.star_info(box=15)

        e = np.nanmedian(ell)
        tht = np.nanmedian(th)
        fw_x = np.nanmedian(fwx)
        fw_y = np.nanmedian(fwy)
        cpe = np.nanmedian(cpe_t)
        mk = np.array(cpe_t) > 10
        n_cpe = len(np.array(cpe_t)[mk])

        t9 = time.time()

        #print(f,hdr["OBSTYPE"],hdr["IMAGETYP"],hdr["OBJECT"],ffs.stats)
        #print(hdr["IMAGETYP"],hdr["OBJECT"],ffs.stats["median"])
        line = 0
        if len(line_val) > 0:
            line = line_val[0]
        #print(f,hdr["IMAGETYP"], hdr["FILTER"], hdr["OBJECT"], len(ffs.coo), ffs.fwhm_x, ffs.fwhm_y, ffs.max_amplitude, ffs.frame_gradient, line, e, se)
        print(f,hdr["IMAGETYP"], hdr["FILTER"], hdr["OBJECT"], (fw_x + fw_y) / 2, e, cpe, n_cpe, ffs.frame_gradient, line)
        #print(t6-t5,t7-t6,t8-t7,t9-t8)

zb08c_1030_55127.fits science z SA95 2.3080933276251323 0.05020142665828087 2.251090993779359 0 19.31313299322767 0
zb08c_1030_55250.fits science B TPHE 6.234942950850645 0.05488745328980249 4.58266014547216 0 0.26529268940379325 109.0
zb08c_1030_55292.fits science V TPHE 6.272471633784436 0.05480124370720668 4.427873807102298 0 0.8743037346904998 109.0
zb08c_1030_55329.fits science Ic TPHE 6.372435926808407 0.05440471348405895 4.221149632176041 0 8.101819653567077 124.0
zb08c_1030_55367.fits science g TPHE 6.437847628122705 0.053134467514820205 2.637536080243247 0 0.36364263727955404 0
zb08c_1030_55404.fits science r TPHE 6.58572132079504 0.05267038223847886 2.223370739498236 0 1.319927836441991 121.0
zb08c_1030_55437.fits science i TPHE 6.468557454392428 0.051879234247964645 2.3259043972381224 0 3.8235416361195576 115.0
zb08c_1030_55474.fits science z TPHE 6.650147596778176 0.05671444403060222 2.3556933222390803 0 17.21413980738589 123.0
zb08c_1030_55587.fits science Ic DN_Aqr 6.2984

In [4]:
print(ffs.stats_description)

{'min': 'Minimal pixel value in the image', 'max': 'Maximum pixel value in the image', 'mean': 'Mean pixel value (arithmetic average)', 'median': 'Median pixel value (robust background estimator)', 'rms': 'Standard deviation of pixel values', 'q_sigma_lower': 'Lower 1-sigma estimate from 15.9% quantile', 'q_sigma_upper': 'Upper 1-sigma estimate from 84.1% quantile', 'q_sigma': 'Robust sigma estimated from 15.9–84.1% quantiles', 'noise': 'Expected total noise (Poisson + read noise)', 'bkg_max_amplitude': 'Maximum amplitude of the background signal across the image, representing the highest estimated background level (in ADU)', 'bkg_frame_gradient': 'Global background gradient across the image frame, describing large-scale background variation (e.g. due to sky brightness gradient, moonlight, or scattered light)', 'sky_surface_coeff': 'Coefficients of the fitted background surface model describing the large-scale sky background variation across the image', 'sky': {'surface_bkg': 'Estimate